# Install packages

In [23]:
!pip install imutils
!pip install opencv-python

!apt-get install graphviz

!pip3 install graphviz==0.5.2
!pip3 install pydot-ng==1.0.0

/bin/sh: apt-get: command not found
/bin/sh: pip3 install graphviz==0.5.2: command not found
/bin/sh: pip3 install pydot-ng==1.0.0: command not found


# Load dataset

In [26]:
from imutils import paths
import os
import numpy as np
from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader


dataset_path = './datasets/plantVillage-full-mohanty'

# Get the list of image names
image_paths = list(paths.list_images(dataset_path))
# Get unique classnames
classNames = [pt.split(os.path.sep)[-2] for pt in image_paths]
classNames = [str(x) for x in np.unique(classNames)]

# Preprocessors
aap = AspectAwarePreprocessor(32, 32)
itap = ImageToArrayPreprocessor()

# Add preprocessors to DataSetLoader class
sdl = SimpleDatasetLoader(preprocessors=[aap, itap])

(data, labels) = sdl.load(image_paths, verbose=500)
data = data.astype('float') / 255.0


[INFO]: Processed 500/54305
[INFO]: Processed 1000/54305
[INFO]: Processed 1500/54305
[INFO]: Processed 2000/54305
[INFO]: Processed 2500/54305
[INFO]: Processed 3000/54305
[INFO]: Processed 3500/54305
[INFO]: Processed 4000/54305
[INFO]: Processed 4500/54305
[INFO]: Processed 5000/54305
[INFO]: Processed 5500/54305
[INFO]: Processed 6000/54305
[INFO]: Processed 6500/54305
[INFO]: Processed 7000/54305
[INFO]: Processed 7500/54305
[INFO]: Processed 8000/54305
[INFO]: Processed 8500/54305
[INFO]: Processed 9000/54305
[INFO]: Processed 9500/54305
[INFO]: Processed 10000/54305
[INFO]: Processed 10500/54305
[INFO]: Processed 11000/54305
[INFO]: Processed 11500/54305
[INFO]: Processed 12000/54305
[INFO]: Processed 12500/54305
[INFO]: Processed 13000/54305
[INFO]: Processed 13500/54305
[INFO]: Processed 14000/54305
[INFO]: Processed 14500/54305
[INFO]: Processed 15000/54305
[INFO]: Processed 15500/54305
[INFO]: Processed 16000/54305
[INFO]: Processed 16500/54305
[INFO]: Processed 17000/54305


# Encode labels and split dataset

In [28]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# Split data into training (75%) and testing (25%) data
(train_x, test_x, train_y, test_y) = train_test_split(data, labels,
test_size=0.25, random_state=42)

# Convert the labels from integers to vectors
lb = LabelBinarizer()
train_y = lb.fit_transform(train_y)
test_y = lb.transform(test_y)


# Initialize AlexNet Model

In [29]:
from utils.models.alexnet import alexnet
from keras.utils import plot_model

model = alexnet()
model.summary()
plot_model(model, to_file="alex.png", show_shapes=True)

# compile model
opt = SGD(lr=0.05)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_28 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_29 (Activation)   (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 6, 6, 384)         885120    
__________

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

# Fit model

In [0]:
# train the network
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY),
              batch_size=32, epochs=100, verbose=1)

model.save('alexnet_model.h5')

# Generate classfication report

In [0]:
from sklearn.metrics import classification_report


# If one hot encoding is used
print(classification_report(test_y, model.predict(test_x),
                            target_names=classNames))


# Plot model

In [0]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 100), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 100), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 100), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 100), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args["output"])
plt.show()